In [1]:
import pandas as pd
import numpy as np

In [2]:
rows = 35000000
data1 = pd.read_csv("1 min traffic intensity 2017.csv", skiprows = range(1, rows + 1))

C:\Users\u180569\AppData\Local\Temp\ipykernel_6752\417819050.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data1 = pd.read_csv("1 min traffic intensity 2017.csv", skiprows = range(1, rows + 1))


In [3]:
stacked_data = data1

# Reset index if needed
#stacked_data.reset_index(drop=True, inplace=True)

# Display the stacked DataFrame
stacked_data.head()

,id_meetlocatie,start_meetperiode,eind_meetperiode,ndw_index,gebruikte_minuten_intensiteit,gebruikte_minuten_snelheid,gem_intensiteit,gem_snelheid,totaal_aantal_rijstroken,rijrichting,technical_exclusion,traffic_flow_deviation_exclusions
0,RWS01_MONIBAS_0271hrr1140ra,2017-06-26 15:22:00,2017-06-26 15:23:00,F4B,1.0,1.0,1680.0,118.0,2,northBound,NaN,NaN
1,RWS01_MONIBAS_0271hrr1140ra,2017-06-26 15:22:00,2017-06-26 15:23:00,F9B,1.0,1.0,840.0,113.0,2,northBound,NaN,NaN
2,RWS01_MONIBAS_0271hrr1140ra,2017-06-26 15:22:00,2017-06-26 15:23:00,F10B,1.0,1.0,120.0,89.0,2,northBound,NaN,NaN
3,RWS01_MONIBAS_0271hrr1140ra,2017-06-26 15:22:00,2017-06-26 15:23:00,F11B,1.0,1.0,240.0,86.0,2,northBound,NaN,NaN
4,RWS01_MONIBAS_0271hrr1140ra,2017-06-26 15:22:00,2017-06-26 15:23:00,F12B,1.0,1.0,1200.0,105.2,2,northBound,NaN,NaN


In [4]:
# Define the list of values to be removed
values_to_remove = ['RWS01_MONIBAS_0071hrl1805ra', 'RWS01_MONIBAS_0281hrl1476ra',
                    'RWS01_MONIBAS_0071hrr1805ra', 'RWS01_MONIBAS_0281hrr1476ra',
                    'RWS01_MONIBAS_0321hrr0654ra', 'RWS01_MONIBAS_0321hrl0654ra',
                    'RWS01_MONIBAS_0321hrr0467ra', 'RWS01_MONIBAS_0321hrl0467ra',
                    'RWS01_MONIBAS_0041hrl0151ra', 'RWS01_MONIBAS_0041hrr0152ra',
                    'RWS01_MONIBAS_0070vwb1401ra', 'RWS01_MONIBAS_0070vwc1401ra'
                   ]

# Filter out rows with specified values in the id_meetlocatie column
stacked_data = stacked_data[~stacked_data['id_meetlocatie'].isin(values_to_remove)]

# Reset index if needed
stacked_data.reset_index(drop=True, inplace=True)

# Display the filtered DataFrame
#filtered_stacked_data.head()

In [5]:
data = stacked_data

In [6]:
# Assuming 'data' is your DataFrame
data['gem_intensiteit'] = data['gem_intensiteit'].replace(-1, 0)
data['gem_snelheid'] = data['gem_snelheid'].replace(-1, 0)

C:\Users\u180569\AppData\Local\Temp\ipykernel_6752\4205887186.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gem_intensiteit'] = data['gem_intensiteit'].replace(-1, 0)
C:\Users\u180569\AppData\Local\Temp\ipykernel_6752\4205887186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gem_snelheid'] = data['gem_snelheid'].replace(-1, 0)


In [7]:
#check for missing values

data.groupby('id_meetlocatie').apply(lambda x: x['gem_snelheid'].isnull().sum())

id_meetlocatie
RWS01_MONIBAS_0271hrr1140ra    0
RWS01_MONIBAS_0281hrl1029ra    0
RWS01_MONIBAS_0281hrr1029ra    0
RWS01_MONIBAS_0731hrl1018ra    0
RWS01_MONIBAS_0731hrr1018ra    0
dtype: int64

In [8]:
data["ndw_index"].unique()

array(['F4B', 'F9B', 'F10B', 'F11B', 'F12B', 'F1B', 'F2B', 'F3B', 'F1A',
       'F3A', 'F5A'], dtype=object)

In [9]:
#remove stopping lane

data = data[data['ndw_index'] != 'F801A']

In [10]:
# Assuming 'data' is your DataFrame and 'time_column' is the name of the column containing the datetime values
data['time_column'] = pd.to_datetime(data['start_meetperiode'])  # Convert to datetime if not already

# Extract year, month, day, hour, and minute into separate columns
data['date'] = data['time_column'].dt.date
data['year'] = data['time_column'].dt.year
data['month'] = data['time_column'].dt.month
data['day'] = data['time_column'].dt.day
data['hour'] = data['time_column'].dt.hour
data['minute'] = data['time_column'].dt.minute
data['day_of_week'] = data['time_column'].dt.dayofweek
data['weekend'] = (data['day_of_week'] >= 5).astype(int)

# Now you have separate columns for year, month, day, hour, and minute

In [11]:
# Add 0.000001 to the values in the "gem_intensiteit" column
data['gem_intensiteit'] += 0.000001

In [12]:
data['intensiteit'] = data['gem_intensiteit']/60

In [13]:
data['sumspeed'] = data['intensiteit'] * data ['gem_snelheid']

In [14]:
#extra check not necessary if previous one was already succesful
#data.groupby('id_meetlocatie').apply(lambda x: x['gem_snelheid'].isnull().sum())

In [15]:
# Select the columns to group by
group_columns = ['id_meetlocatie', 'date', 'year', 'month', 'day', 'hour', 'minute', 'day_of_week', 'weekend', 'totaal_aantal_rijstroken']

# Group by the selected columns and aggregate other columns
grouped_data = data.groupby(group_columns).agg({'intensiteit': 'sum', 'sumspeed': 'sum'}).reset_index()

In [16]:
grouped_data['avg_speed'] = grouped_data['sumspeed'] / grouped_data['intensiteit']
grouped_data = grouped_data.drop(columns=['sumspeed'])

In [17]:
grouped_data.head(20)

,id_meetlocatie,date,year,month,day,hour,minute,day_of_week,weekend,totaal_aantal_rijstroken,intensiteit,avg_speed
0,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,22,0,0,2,68.0,110.470588
1,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,23,0,0,2,66.0,116.818333
2,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,24,0,0,2,38.0,113.474210
3,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,25,0,0,2,62.0,126.580645
4,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,26,0,0,2,58.0,128.413793
5,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,27,0,0,2,48.0,121.042500
6,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,28,0,0,2,30.0,119.666000
7,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,29,0,0,2,62.0,112.517581
8,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,30,0,0,2,86.0,124.629070
9,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,31,0,0,2,62.0,121.645161


In [18]:
grouped_data.shape

(2334165, 12)

In [19]:
'''test to see whether null variables'''
#grouped_data.groupby('id_meetlocatie').apply(lambda x: x['avg_speed'].isnull().sum())

'test to see whether null variables'

In [20]:
#grouped_data['hour'] = pd.to_numeric(grouped_data['hour'], errors='coerce')

In [21]:
#grouped_data = grouped_data[(grouped_data['hour'] > 5)] 

In [22]:
#grouped_data = grouped_data[(grouped_data['hour'] < 21)]

In [23]:
holidays = pd.read_csv("holidays.csv")

In [24]:
import pandas as pd

# Assuming holidays is your DataFrame and 'date' is the column containing dates
# Convert 'date' column to datetime object
holidays['date'] = pd.to_datetime(holidays['date'], errors='coerce')

# Check unique values and infer date format
unique_dates = holidays['date'].dropna().dt.strftime('%Y-%m-%d').unique()
date_format = unique_dates[0] if len(unique_dates) == 1 else "Multiple formats detected"

# Print the inferred date format
print("Inferred date format:", date_format)

Inferred date format: Multiple formats detected


In [25]:
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')

# Check unique values and infer date format
unique_dates = grouped_data['date'].dropna().dt.strftime('%Y-%m-%d').unique()
date_format = unique_dates[0] if len(unique_dates) == 1 else "Multiple formats detected"

# Print the inferred date format
print("Inferred date format:", date_format)

Inferred date format: Multiple formats detected


In [26]:
# Round the 'intensiteit' and 'alg_speed' columns to two decimals
grouped_data['intensiteit'] = grouped_data['intensiteit'].round(2)
grouped_data['avg_speed'] = grouped_data['avg_speed'].round(2)


In [27]:
merged_data = pd.merge(grouped_data, holidays, on='date', how='left')

merged_data['is_public_holiday'] = merged_data['is_public_holiday'].fillna(0).astype(int)

In [28]:
merged_data.head(30)

,id_meetlocatie,date,year,month,day,hour,minute,day_of_week,weekend,totaal_aantal_rijstroken,intensiteit,avg_speed,is_public_holiday
0,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,22,0,0,2,68.0,110.47,0
1,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,23,0,0,2,66.0,116.82,0
2,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,24,0,0,2,38.0,113.47,0
3,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,25,0,0,2,62.0,126.58,0
4,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,26,0,0,2,58.0,128.41,0
5,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,27,0,0,2,48.0,121.04,0
6,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,28,0,0,2,30.0,119.67,0
7,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,29,0,0,2,62.0,112.52,0
8,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,30,0,0,2,86.0,124.63,0
9,RWS01_MONIBAS_0271hrr1140ra,2017-06-26,2017,6,26,15,31,0,0,2,62.0,121.65,0


In [29]:
# Assuming your DataFrame is named merged_data
num_zeros = (merged_data['avg_speed'] == 0.0).sum()
print("Number of zero values in the 'avg_speed' column:", num_zeros)

Number of zero values in the 'avg_speed' column: 101374


In [30]:
merged_data.to_csv('Preprocessed_1min_2017_2.csv', index=False)